In [ ]:
!pip install transformers datasets


In [1]:
from transformers import pipeline, set_seed, GPT2LMHeadModel, AutoTokenizer, TextDataset, AutoModelForCausalLM
import datasets
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer, EarlyStoppingCallback
import numpy as np

In [2]:
model = AutoModelForCausalLM.from_pretrained("imthanhlv/vigpt2medium")
tokenizer = AutoTokenizer.from_pretrained("imthanhlv/vigpt2medium")
special_tokens_dict = {'additional_special_tokens': ['<|beginofdes|>','<|endofdes|>', '<br>']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))
tokenizer.pad_token = tokenizer.eos_token

In [3]:
len(tokenizer)

50261

In [4]:
model.config

GPT2Config {
  "_name_or_path": "imthanhlv/vigpt2medium",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.0,
  "bos_token_id": 50256,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_special_tokens": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.0,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 250
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.30.1",
  "use_cache": true,
  "vocab_size": 50261
}

In [3]:
tokenizer

GPT2TokenizerFast(name_or_path='imthanhlv/vigpt2medium', vocab_size=50257, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|beginofdes|>', '<|endofdes|>', '<br>']}, clean_up_tokenization_spaces=True)

In [4]:
text = "<|beginofdes|>ngoc dang<|beginofdes|>di<|endofdes|>và<br>ăn chơi"

In [5]:
tokenizer.encode(text)


[50258, 46476, 6689, 50258, 4087, 50259, 7061, 50260, 522, 1391]

In [6]:
tokenizer.decode([50258, 46476, 6689, 50258, 4087, 50259, 7061, 50260, 522, 1391])


'<|beginofdes|>ngoc dang<|beginofdes|>di<|endofdes|>và<br>ăn chơi'

In [7]:
def get_text_ds(path):
    lines = []
    text = ''
    with open(path) as f:
        for line in f:
            if line.find('xxxxxEndxxxxx') != -1:
                if text.strip():
                    lines.append(text.strip())
                    text = ''
                else:
                    continue
            else:
                text = text + line
    vi_ds = datasets.Dataset.from_dict({'text': lines})
    return vi_ds

In [8]:
train_ds = get_text_ds('fashion_15_10_2022_train.txt')
test_ds = get_text_ds('fashion_15_10_2022_test.txt')

In [9]:
train_ds


Dataset({
    features: ['text'],
    num_rows: 40803
})

In [10]:
test_ds


Dataset({
    features: ['text'],
    num_rows: 10201
})

In [11]:
context_length = 512
def tokenize(batch):
  outputs = tokenizer(
      batch['text'],
      truncation=True,
      max_length=context_length,
      return_overflowing_tokens=True,
      return_length=True
      #padding='max_length'
  )
  input_batch = []
  for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
      input_batch.append(input_ids)
  result = {}
  result['input_ids'] = input_batch
  # result["labels"] = result["input_ids"].copy()
  return result


tokenized_train_datasets = train_ds.map(
    tokenize, batched=True, remove_columns=['text']
)

Map:   0%|          | 0/40803 [00:00<?, ? examples/s]

In [ ]:
tokenized_train_datasets

Dataset({
    features: ['input_ids'],
    num_rows: 18180
})

In [12]:
tokenized_test_datasets = test_ds.map(
    tokenize, batched=True, remove_columns=['text']
)

Map:   0%|          | 0/10201 [00:00<?, ? examples/s]

In [ ]:
tokenized_test_datasets


Dataset({
    features: ['input_ids'],
    num_rows: 10201
})

In [13]:
tokenizer.decode(tokenized_train_datasets[120]['input_ids'])


'sơ mi<|beginofdes|>những chiếc đầm xếp li dáng maxi, quần ống rộng, sơ mi oversize,... đó mới thực sự là các item mang lại cảm giác thời thượng cũng như giúp các tín đồ khoe khéo tài lên đồ của mình.<|endofdes|>'

In [14]:
tokenizer.decode(tokenized_test_datasets[120]['input_ids'])


'trang phục<|beginofdes|>không phải ngẫu nhiên mà các người đẹp hoàng gia hay các đệ nhất phu nhân thường ưu ái những trang phục vải tweed, đặc biệt là các nhà mốt cũng thường sáng tạo nên những thiết kế mang chất liệu vải sang chảnh này. lý do là bởi vải tweed thực sự có thể khiến vẻ ngoài của người diện trông thượng lưu và giàu có hơn.<|endofdes|>'

In [15]:
train_data_path = 'article_512/train'
tokenized_train_datasets.save_to_disk(train_data_path)

test_data_path = 'article_512/test'
tokenized_test_datasets.save_to_disk(test_data_path)

Saving the dataset (0/1 shards):   0%|          | 0/40805 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10201 [00:00<?, ? examples/s]